# Finding Good Background Pixels in DES Data

We're gonna go through some of the basics on how to access good pixels which are also background pixels in DES data.

In [1]:
import os
import fitsio
import yaml
import esutil
import numpy as np

import matplotlib.pyplot as plt

## What is a good background pixel anyways?

We will define the good background pixels as

1. not associated with any detected objects
2. positive weight
3. no bit flags set

Let's discuss these in more detail.

### How can I tell if a pixel is associated with any detected objects?

For this task, we will use the segmentation maps associated with the coadd and single-epoch images. Remember that 
a segementation image has integer values and marks areas of the image associated with each detection. For our purposes, we only want pixels where the segmentation map is 0.

### What does it mean for a pixel to have positive weight?

Remember that the weight map is the inverse of the variance of the data in the pixel. This format means that pixels with large variance have very small weights. Sometimes, we set the value of the weight map to zero by hand in order to indicate that a given pixel should be ignored. Thus we want to demand that the weight map is greater than zero.

### What does it mean for a pixel to have no bit flags set?

As images are processed, numerical operations are performed on the pixels and sets of pixels are flagged as being from artifacts, defects, etc. This information is typically stored in what is known as a bit mask image. A bit mask image is an image of integers. For each pixel, the underlying binary representation of the number is used to store flags indicating different meanings. If not bit flags are set, then value of this field should be 0.

## How does this apply to DES data?

For DES data, we will demand these conditions of both the single-epoch image and the coadd image in the same part of the sky. To do this we will have to map all of the single-epoch pixels to their nearest location in the coadd image. 

Below I've put some code in the help guide you through this task. The steps will be as follows.

1. Read in the coadd image and single-epoch image data.
2. Map all of the single-epoch pixels to their nearest coadd pixel.
4. Make the proper set of cuts on all of these quantities. 
5. Visualize the resulting image.

### 1. Read in the Image Data

We'll use the wcs reading function from the last tutorial.

In [11]:
def read_wcs(pth, ext=0):
    hdr = fitsio.read_header(pth, ext=ext)
    dct = {}
    for k in hdr.keys():
        try:
            dct[k.lower()] = hdr[k]
        except Exception:
            pass
    return esutil.wcsutil.WCS(dct)

Now let's get the images and the WCS solutions. First we grab the info dictionary from the YAML to get the paths.

In [2]:
meds_dir = "/cosmo/scratch/mrbecker/MEDS_DIR"
tilename = "DES0124-3332"
band = "i"
yaml_pth = os.path.join(
    meds_dir, 
    "des-pizza-slices-y6-v8/pizza_cutter_info/%s_%s_pizza_cutter_info.yaml" % (
        tilename, band
    )
)

with open(yaml_pth, "r") as fp:
    info = yaml.safe_load(fp.read())

In [10]:
import os

meds_dir = os.environ.get("MEDS_DIR")
tilename = "DES2359-6331"
band = "i"
yaml_pth = os.path.join(
    meds_dir, 
    "des-pizza-slices-y6-v6/pizza_cutter_info/%s_%s_pizza_cutter_info.yaml" % (
        tilename, band
    )
)

with open(yaml_pth, "r") as fp:
    info = yaml.safe_load(fp.read())

And we read the stuff:

In [15]:
coadd_wcs = read_wcs(info['image_path'], ext=info['image_ext'])
coadd_image = fitsio.read(info['image_path'], ext=info['image_ext'])
coadd_weight = read_wcs(info['weight_path'], ext=info['weight_ext'])
coadd_bmask = read_wcs(info['bmask_path'], ext=info['bmask_ext'])
coadd_seg = read_wcs(info['seg_path'], ext=info['seg_ext'])

We will look at the 5th single-epoch image.

In [18]:
se_ind = 4  # 5th image is index 4
si = info['src_info'][se_ind]
se_wcs = read_wcs(si['image_path'], ext=si['image_ext'])
se_image = fitsio.read(si['image_path'], ext=si['image_ext'])
se_weight = fitsio.read(si['weight_path'], ext=si['weight_ext'])
se_bmask = fitsio.read(si['bmask_path'], ext=si['bmask_ext'])

### 2. Map all of the single-epoch pixels to the nearest coadd pixel

To get you started, I have build a list of the pixel indices for the single-epoch image pixels below.

In [22]:
xind, yind = np.meshgrid(se_image.shape[1], se_image.shape[0])
xind = xind.ravel()
yind = yind.ravel()

In [ ]:
# do the rest here, computing the coadd indices for each single-epoch pixel
# when you do this, you'll need to cut out indices less than zero or greater than or equal to the dimenions
# this can be done with mask arrays
#  msk = (x >= 0) & (x < coadd_image.shape[1])
#  x = x[msk]

### 3. Make the cuts on the proper quantities

Once you have the pixel locations remapped, then you can make cuts using the same mask array syntax as above.

### 4. Visualize the Data

Make a plot of the good and bad pixels in the image.